<a href="https://colab.research.google.com/github/Patsonu63/Semantic-Similarity-with-KerasNLP.ipynb/blob/main/Semantic_Similarity_with_KerasNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Description**: Use pretrained models from KerasNLP for the Semantic Similarity Task.



In [3]:
!pip install -q --upgrade keras-nlp
!pip install -q --upgrade keras  # Upgrade to Keras 3.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.8/571.8 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 601.3/601.3 MB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 102.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 95.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 99.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 73.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.7/347.7 kB 42.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.15.1 requires tensorflow<2.16,>=2.15, but you have tensorflow 2.17.0 which is incompatible.


In [4]:
import numpy as np
import tensorflow as tf
import keras
import keras_nlp
import tensorflow_datasets as tfds

In [5]:
snli_train = tfds.load("snli", split="train[:20%]")
snli_val = tfds.load("snli", split="validation")
snli_test = tfds.load("snli", split="test")

# Here's an example of how our training samples look like, where we randomly select
# four samples:
sample = snli_test.batch(4).take(1).get_single_element()
sample

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating test examples...:   0%|          | 0/10000 [00:00<?, ? examples/s]

...

Shuffling /root/tensorflow_datasets/snli/incomplete.48SPIU_1.1.0/snli-validation.tfrecord*...:   0%|          …

Generating train examples...:   0%|          | 0/550152 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/snli/incomplete.48SPIU_1.1.0/snli-train.tfrecord*...:   0%|          | 0/5…

Dataset snli downloaded and prepared to /root/tensorflow_datasets/snli/1.1.0. Subsequent calls will reuse this data.


{'hypothesis': <tf.Tensor: shape=(4,), dtype=string, numpy=
 array([b'A girl is entertaining on stage',
        b'A group of people posing in front of a body of water.',
        b"The group of people aren't inide of the building.",
        b'The people are taking a carriage ride.'], dtype=object)>,
 'label': <tf.Tensor: shape=(4,), dtype=int64, numpy=array([0, 0, 0, 0])>,
 'premise': <tf.Tensor: shape=(4,), dtype=string, numpy=
 array([b'A girl in a blue leotard hula hoops on a stage with balloon shapes in the background.',
        b'A group of people taking pictures on a walkway in front of a large body of water.',
        b'Many people standing outside of a place talking to each other in front of a building that has a sign that says "HI-POINTE."',
        b'Three people are riding a carriage pulled by four horses.'],
       dtype=object)>}

In [6]:
def filter_labels(sample):
    return sample["label"] >= 0

In [7]:

def split_labels(sample):
    x = (sample["hypothesis"], sample["premise"])
    y = sample["label"]
    return x, y


train_ds = (
    snli_train.filter(filter_labels)
    .map(split_labels, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(16)
)
val_ds = (
    snli_val.filter(filter_labels)
    .map(split_labels, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(16)
)
test_ds = (
    snli_test.filter(filter_labels)
    .map(split_labels, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(16)
)


In [8]:
bert_classifier = keras_nlp.models.BertClassifier.from_preset(
    "bert_tiny_en_uncased", num_classes=3
)

100%|██████████| 139/139 [00:00<00:00, 125kB/s]


100%|██████████| 507/507 [00:00<00:00, 449kB/s]


100%|██████████| 16.8M/16.8M [00:00<00:00, 27.6MB/s]


100%|██████████| 547/547 [00:00<00:00, 344kB/s]


100%|██████████| 226k/226k [00:00<00:00, 1.22MB/s]


In [9]:
bert_classifier.fit(train_ds, validation_data=val_ds, epochs=1)

   6867/Unknown 178s 23ms/step - loss: 0.8909 - sparse_categorical_accuracy: 0.5729

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


6867/6867 ━━━━━━━━━━━━━━━━━━━━ 191s 25ms/step - loss: 0.8909 - sparse_categorical_accuracy: 0.5729 - val_loss: 0.5894 - val_sparse_categorical_accuracy: 0.7590


In [10]:
bert_classifier.evaluate(test_ds)

614/614 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - loss: 0.5789 - sparse_categorical_accuracy: 0.7663


[0.5900533199310303, 0.7623167634010315]

In [11]:
bert_classifier = keras_nlp.models.BertClassifier.from_preset(
    "bert_tiny_en_uncased", num_classes=3
)
bert_classifier.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(5e-5),
    metrics=["accuracy"],
)

bert_classifier.fit(train_ds, validation_data=val_ds, epochs=1)
bert_classifier.evaluate(test_ds)

6867/6867 ━━━━━━━━━━━━━━━━━━━━ 191s 25ms/step - accuracy: 0.5974 - loss: 0.8671 - val_accuracy: 0.7626 - val_loss: 0.5818
614/614 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.7686 - loss: 0.5772


[0.5848731398582458, 0.7660830616950989]

In [12]:
class TriangularSchedule(keras.optimizers.schedules.LearningRateSchedule):
    """Linear ramp up for `warmup` steps, then linear decay to zero at `total` steps."""

    def __init__(self, rate, warmup, total):
        self.rate = rate
        self.warmup = warmup
        self.total = total

    def get_config(self):
        config = {"rate": self.rate, "warmup": self.warmup, "total": self.total}
        return config

    def __call__(self, step):
        step = keras.ops.cast(step, dtype="float32")
        rate = keras.ops.cast(self.rate, dtype="float32")
        warmup = keras.ops.cast(self.warmup, dtype="float32")
        total = keras.ops.cast(self.total, dtype="float32")

        warmup_rate = rate * step / self.warmup
        cooldown_rate = rate * (total - step) / (total - warmup)
        triangular_rate = keras.ops.minimum(warmup_rate, cooldown_rate)
        return keras.ops.maximum(triangular_rate, 0.0)


bert_classifier = keras_nlp.models.BertClassifier.from_preset(
    "bert_tiny_en_uncased", num_classes=3
)

# Get the total count of training batches.
# This requires walking the dataset to filter all -1 labels.
epochs = 3
total_steps = sum(1 for _ in train_ds.as_numpy_iterator()) * epochs
warmup_steps = int(total_steps * 0.2)

bert_classifier.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.AdamW(
        TriangularSchedule(1e-4, warmup_steps, total_steps)
    ),
    metrics=["accuracy"],
)

bert_classifier.fit(train_ds, validation_data=val_ds, epochs=epochs)

Epoch 1/3
6867/6867 ━━━━━━━━━━━━━━━━━━━━ 201s 27ms/step - accuracy: 0.5338 - loss: 0.9328 - val_accuracy: 0.7600 - val_loss: 0.5851
Epoch 2/3
6867/6867 ━━━━━━━━━━━━━━━━━━━━ 167s 24ms/step - accuracy: 0.7306 - loss: 0.6503 - val_accuracy: 0.7887 - val_loss: 0.5364
Epoch 3/3
6867/6867 ━━━━━━━━━━━━━━━━━━━━ 164s 24ms/step - accuracy: 0.7709 - loss: 0.5696 - val_accuracy: 0.7935 - val_loss: 0.5207


In [13]:
bert_classifier.evaluate(test_ds)

614/614 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.7931 - loss: 0.5166


[0.526474118232727, 0.7886807918548584]

In [14]:
bert_classifier.save("bert_classifier.keras")
restored_model = keras.models.load_model("bert_classifier.keras")
restored_model.evaluate(test_ds)

/usr/local/lib/python3.10/dist-packages/keras/src/saving/serialization_lib.py:734: UserWarning: `compile()` was not called as part of model loading because the model's `compile()` method is custom. All subclassed Models that have `compile()` overridden should also override `get_compile_config()` and `compile_from_config(config)`. Alternatively, you can call `compile()` manually after loading.
  instance.compile_from_config(compile_config)
/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 83 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


614/614 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - loss: 0.5166 - sparse_categorical_accuracy: 0.7931


[0.526474118232727, 0.7886807918548584]

In [15]:
# Convert to Hypothesis-Premise pair, for forward pass through model
sample = (sample["hypothesis"], sample["premise"])
sample

(<tf.Tensor: shape=(4,), dtype=string, numpy=
 array([b'A girl is entertaining on stage',
        b'A group of people posing in front of a body of water.',
        b"The group of people aren't inide of the building.",
        b'The people are taking a carriage ride.'], dtype=object)>,
 <tf.Tensor: shape=(4,), dtype=string, numpy=
 array([b'A girl in a blue leotard hula hoops on a stage with balloon shapes in the background.',
        b'A group of people taking pictures on a walkway in front of a large body of water.',
        b'Many people standing outside of a place talking to each other in front of a building that has a sign that says "HI-POINTE."',
        b'Three people are riding a carriage pulled by four horses.'],
       dtype=object)>)

In [16]:
predictions = bert_classifier.predict(sample)


def softmax(x):
    return np.exp(x) / np.exp(x).sum(axis=0)


# Get the class predictions with maximum probabilities
predictions = softmax(predictions)

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step


In [ ]:
# Inittializing a RoBERTa from preset
roberta_classifier = keras_nlp.models.RobertaClassifier.from_preset(
    "roberta_base_en", num_classes=3
)

roberta_classifier.fit(train_ds, validation_data=val_ds, epochs=1)

roberta_classifier.evaluate(test_ds)

100%|██████████| 141/141 [00:00<00:00, 137kB/s]


100%|██████████| 498/498 [00:00<00:00, 434kB/s]


100%|██████████| 474M/474M [00:26<00:00, 18.4MB/s]


100%|██████████| 463/463 [00:00<00:00, 369kB/s]


100%|██████████| 0.99M/0.99M [00:00<00:00, 3.04MB/s]


100%|██████████| 446k/446k [00:00<00:00, 1.79MB/s]


   6621/Unknown 9464s 1s/step - loss: 1.1015 - sparse_categorical_accuracy: 0.3404

In [ ]:
predictions = roberta_classifier.predict(sample)
print(tf.math.argmax(predictions, axis=1).numpy())